<a href="https://colab.research.google.com/github/gbessardon/Create_plots/blob/main/open_LUCAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relevant litterature
## LUCAS Copernicus 2018: Earth Observation relevant in-situ data on land cover throughout the European Union paper
https://essd.copernicus.org/preprints/essd-2020-178/essd-2020-178.pdf




## csv labels description
https://ec.europa.eu/eurostat/documents/205002/8072634/LUCAS2018-RecordDescriptor-190611.pdf

## C3 classification Land cover & land use

https://ec.europa.eu/eurostat/documents/205002/8072634/LUCAS2018-C3-Classification.pdf

## Technical reference document C5 Plant Identification Guide

https://ec.europa.eu/eurostat/documents/205002/8072634/LUCAS2018-C5-PlantIdentificationGuide.pdf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from osgeo import gdal

In [ ]:
!pip install geopandas
!pip install pyproj

     |████████████████████████████████| 1.0MB 4.3MB/s 
     |████████████████████████████████| 14.8MB 284kB/s 
     |████████████████████████████████| 6.5MB 41.4MB/s 


In [ ]:
import geopandas as gpd
import numpy as np

In [ ]:
import shapely.geometry as geom

In [ ]:
fn='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_polygons.shp'

In [ ]:
import pandas as pd
from pyproj import Proj

In [ ]:
latmin=50
latmax=56
lonmin=-11
lonmax=-4
p1 = geom.Point(lonmin,latmin)
p2 = geom.Point(lonmax,latmin)
p3 = geom.Point(lonmax,latmax)
p4 = geom.Point(lonmin,latmax)

pointList = [p1, p2, p3, p4, p1]

Rectangle=geom.Polygon([[p.x, p.y] for p in pointList])

In [ ]:
gdf = gpd.read_file(fn)

In [ ]:
fgdf=gdf.loc[gdf.within(Rectangle)==True]

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_attributes.csv')
fildf=df.iloc[gdf.loc[gdf.within(Rectangle)==True].index]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
import numpy as np
cover_list=np.unique(fildf.LC1)
numbers=[np.where(f==cover_list)[0][0] for f in fildf.LC1]

In [ ]:
fgdf['values']=numbers

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
ds=gdal.Open('/content/drive/MyDrive/CORINE/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif')
p=Proj(ds.GetProjection())
ULX,ULY=p(lonmin,latmax)
LRX,LRY=p(lonmax,latmin)
opt=gdal.WarpOptions(xRes=10,yRes=10,outputBounds=(ULX, LRY, LRX, ULY),srcSRS=p.to_wkt())
gdal.Warp('/content/smallcor.tif',ds,options=opt)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f900fcdd8a0> >

In [ ]:
help(gdal.WarpOptions)

Help on function WarpOptions in module osgeo.gdal:

WarpOptions(options=[], format='GTiff', outputBounds=None, outputBoundsSRS=None, xRes=None, yRes=None, targetAlignedPixels=False, width=0, height=0, srcSRS=None, dstSRS=None, srcAlpha=False, dstAlpha=False, warpOptions=None, errorThreshold=None, warpMemoryLimit=None, creationOptions=None, outputType=0, workingType=0, resampleAlg=None, srcNodata=None, dstNodata=None, multithread=False, tps=False, rpc=False, geoloc=False, polynomialOrder=None, transformerOptions=None, cutlineDSName=None, cutlineLayer=None, cutlineWhere=None, cutlineSQL=None, cutlineBlend=None, cropToCutline=False, copyMetadata=True, metadataConflictValue=None, setColorInterpretation=False, callback=None, callback_data=None)
    Create a WarpOptions() object that can be passed to gdal.Warp()
    Keyword arguments are :
      options --- can be be an array of strings, a string or let empty and filled from other keywords.
      format --- output format ("GTiff", etc...)
  

In [ ]:
dsc=gdal.Open('/content/smallcor.tif')
#gdal.Warp('smallcorlatlon.tif',dsc,dstSRS='EPSG:4326')

In [ ]:
Wopt=gdal.WarpOptions(srcSRS=ds.GetProjectionRef(), dstSRS=dstcrs,xRes=100,yRes=100,creationOptions=gdal.ParseCommandLine("-co COMPRESS=PACKBITS"))

'CONVERSION["PROJ-based coordinate operation",METHOD["PROJ-based operation method: +proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs"]]'